In [1]:
import pandas as pd
data = pd.read_csv('output.csv', error_bad_lines=False);
data_text = data[['Page Title']]
data_text['index'] = data_text.index
documents = data_text

In [2]:
print(len(documents))
print(documents[:5])

33686
                                          Page Title  index
0  \n            Pôle-Emploi : les différentes ca...      0
1  Ne pas déclarer ses revenus à pôle emploi [Rés...      1
2  \n            AAH 2019 : montant et plafonds d...      2
3  \n            Allocation de rentrée scolaire (...      3
4  \nPrise d'otages à Blagnac : l'homme retranché...      4


In [3]:
import gensim
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/Phoenix/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
stemmer = SnowballStemmer("french")
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

NameError: name 'sentence' is not defined

In [7]:
from nltk.stem.porter import *
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['Papillon', ':', 'coloriages']


 tokenized and lemmatized document: 
['papillon', 'coloriag']


In [12]:
documents = documents.dropna(subset=['Page Title'])
processed_docs = documents['Page Title'].map(preprocess)
processed_docs[:10]

0           [pôl, emploi, différent, catégor, chômeur]
1         [déclar, revenus, pôl, emploi, résolu, ferm]
2                            [mont, plafond, ressourc]
3                              [alloc, rentr, scolair]
4    [pris, otag, blagnac, homm, retranch, ouvr, po...
5                                    [dessert, maison]
6                                     [meteo, bordeau]
7                              [horoscop, chien, jour]
8                            [horoscop, chien, demain]
9                                     [programm, soir]
Name: Page Title, dtype: object

In [13]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 catégor
1 chômeur
2 différent
3 emploi
4 pôl
5 déclar
6 ferm
7 revenus
8 résolu
9 mont
10 plafond


In [14]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [15]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(398, 1), (399, 1), (1316, 1)]

In [16]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 398 ("coloriag") appears 1 time.
Word 399 ("imprim") appears 1 time.
Word 1316 ("lapin") appears 1 time.


In [17]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.7612892254940364), (1, 0.648412457581353)]


In [18]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [19]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.023*"pour" + 0.016*"rob" + 0.014*"franc" + 0.012*"sous" + 0.012*"kat" + 0.011*"jam" + 0.011*"san" + 0.011*"demand" + 0.010*"tres" + 0.010*"philipp"
Topic: 1 
Words: 0.083*"résolu" + 0.082*"ferm" + 0.053*"comment" + 0.027*"résultat" + 0.021*"dan" + 0.015*"pour" + 0.010*"vent" + 0.010*"radar" + 0.009*"fair" + 0.009*"juin"
Topic: 2 
Words: 0.038*"macron" + 0.028*"meghan" + 0.025*"pour" + 0.024*"markl" + 0.023*"brigitt" + 0.022*"princ" + 0.019*"emmanuel" + 0.017*"harry" + 0.016*"mort" + 0.015*"franc"
Topic: 3 
Words: 0.034*"avec" + 0.028*"toujour" + 0.025*"amour" + 0.025*"jean" + 0.023*"linternaut" + 0.020*"star" + 0.016*"bel" + 0.015*"michel" + 0.015*"pierr" + 0.013*"candidat"
Topic: 4 
Words: 0.024*"recet" + 0.022*"enfant" + 0.021*"pour" + 0.019*"avant" + 0.016*"laur" + 0.016*"per" + 0.015*"smet" + 0.014*"terr" + 0.014*"coloriag" + 0.013*"paris"
Topic: 5 
Words: 0.028*"coup" + 0.020*"cuisin" + 0.017*"carr" + 0.017*"chez" + 0.015*"être" + 0.014*"trump" + 0.014*"dan" + 0

In [26]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
lda_model_tfidf.save('model100.gensim')
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.019*"résolu" + 0.017*"ferm" + 0.016*"macron" + 0.016*"brigitt" + 0.012*"cuisin" + 0.011*"coloriag" + 0.010*"château" + 0.010*"cart" + 0.010*"comment" + 0.010*"mar"
Topic: 1 Word: 0.012*"johnny" + 0.011*"mort" + 0.010*"franc" + 0.010*"hallyday" + 0.009*"pierr" + 0.008*"michel" + 0.008*"plus" + 0.008*"pass" + 0.008*"comment" + 0.007*"résolu"
Topic: 2 Word: 0.022*"recet" + 0.014*"avec" + 0.013*"plus" + 0.013*"pour" + 0.012*"bel" + 0.011*"meilleur" + 0.010*"résolu" + 0.009*"ferm" + 0.008*"imag" + 0.008*"suit"
Topic: 3 Word: 0.018*"météo" + 0.010*"résolu" + 0.010*"vent" + 0.010*"ferm" + 0.009*"avec" + 0.009*"terr" + 0.009*"jean" + 0.008*"pomm" + 0.008*"pour" + 0.007*"mond"
Topic: 4 Word: 0.012*"chez" + 0.009*"avec" + 0.009*"pour" + 0.008*"meghan" + 0.008*"markl" + 0.008*"hallyday" + 0.008*"san" + 0.007*"françois" + 0.007*"moin" + 0.007*"laetici"
Topic: 5 Word: 0.011*"franc" + 0.010*"kat" + 0.009*"princ" + 0.009*"meghan" + 0.008*"recet" + 0.008*"harry" + 0.008*"middleton" + 

In [21]:
processed_docs[4310]

['papillon', 'coloriag']

In [22]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.7749691009521484	 
Topic: 0.024*"recet" + 0.022*"enfant" + 0.021*"pour" + 0.019*"avant" + 0.016*"laur" + 0.016*"per" + 0.015*"smet" + 0.014*"terr" + 0.014*"coloriag" + 0.013*"paris"

Score: 0.02500925399363041	 
Topic: 0.026*"fill" + 0.023*"pour" + 0.021*"vous" + 0.019*"votr" + 0.017*"fait" + 0.016*"avec" + 0.014*"plus" + 0.013*"françois" + 0.013*"sarkozy" + 0.011*"nicol"

Score: 0.02500304952263832	 
Topic: 0.034*"avec" + 0.028*"toujour" + 0.025*"amour" + 0.025*"jean" + 0.023*"linternaut" + 0.020*"star" + 0.016*"bel" + 0.015*"michel" + 0.015*"pierr" + 0.013*"candidat"

Score: 0.025002889335155487	 
Topic: 0.083*"résolu" + 0.082*"ferm" + 0.053*"comment" + 0.027*"résultat" + 0.021*"dan" + 0.015*"pour" + 0.010*"vent" + 0.010*"radar" + 0.009*"fair" + 0.009*"juin"

Score: 0.02500271238386631	 
Topic: 0.035*"météo" + 0.034*"port" + 0.027*"grand" + 0.021*"franc" + 0.019*"cet" + 0.018*"vill" + 0.016*"saint" + 0.016*"mer" + 0.015*"dan" + 0.014*"camill"

Score: 0.025002630427479744	 


In [23]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.7749522924423218	 
Topic: 0.023*"ferm" + 0.021*"résolu" + 0.013*"coloriag" + 0.012*"pour" + 0.009*"quel" + 0.009*"coup" + 0.008*"franc" + 0.008*"christin" + 0.008*"main" + 0.008*"comment"

Score: 0.025013919919729233	 
Topic: 0.020*"mort" + 0.019*"carr" + 0.017*"emmanuel" + 0.013*"comment" + 0.012*"macron" + 0.011*"coup" + 0.011*"nicol" + 0.009*"long" + 0.009*"résolu" + 0.009*"ferm"

Score: 0.02500595711171627	 
Topic: 0.014*"françois" + 0.013*"famill" + 0.010*"michel" + 0.009*"vous" + 0.009*"per" + 0.009*"chemin" + 0.008*"octobr" + 0.007*"sous" + 0.007*"autr" + 0.007*"frais"

Score: 0.025004690513014793	 
Topic: 0.015*"miss" + 0.012*"terr" + 0.012*"recet" + 0.011*"plus" + 0.011*"chez" + 0.011*"fair" + 0.010*"résolu" + 0.010*"trump" + 0.010*"apres" + 0.010*"pomm"

Score: 0.025004291906952858	 
Topic: 0.023*"météo" + 0.022*"port" + 0.019*"maison" + 0.015*"rob" + 0.014*"nouvel" + 0.013*"cuisin" + 0.012*"vent" + 0.011*"château" + 0.010*"paris" + 0.010*"pour"

Score: 0.0250041931

In [24]:
unseen_document = 'Meghan, épouse du prince Harry, a donné naissance à un garçon'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.38253113627433777	 Topic: 0.038*"macron" + 0.028*"meghan" + 0.025*"pour" + 0.024*"markl" + 0.023*"brigitt"
Score: 0.28079739212989807	 Topic: 0.026*"fill" + 0.023*"pour" + 0.021*"vous" + 0.019*"votr" + 0.017*"fait"
Score: 0.24913467466831207	 Topic: 0.035*"météo" + 0.034*"port" + 0.027*"grand" + 0.021*"franc" + 0.019*"cet"
Score: 0.01251250971108675	 Topic: 0.028*"coup" + 0.020*"cuisin" + 0.017*"carr" + 0.017*"chez" + 0.015*"être"
Score: 0.012510773725807667	 Topic: 0.083*"résolu" + 0.082*"ferm" + 0.053*"comment" + 0.027*"résultat" + 0.021*"dan"
Score: 0.012503588572144508	 Topic: 0.034*"avec" + 0.028*"toujour" + 0.025*"amour" + 0.025*"jean" + 0.023*"linternaut"
Score: 0.01250299159437418	 Topic: 0.023*"pour" + 0.016*"rob" + 0.014*"franc" + 0.012*"sous" + 0.012*"kat"
Score: 0.012502348981797695	 Topic: 0.032*"femm" + 0.030*"hallyday" + 0.029*"meteo" + 0.029*"pour" + 0.025*"johnny"
Score: 0.012502342462539673	 Topic: 0.020*"dan" + 0.020*"mond" + 0.019*"avec" + 0.018*"pour" + 0.

In [28]:
import pyLDAvis
import pyLDAvis.gensim as gensimvis
lda100 = gensim.models.ldamodel.LdaModel.load('model100.gensim')
lda_display100 = pyLDAvis.gensim.prepare(lda100, bow_corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display100)

/Users/Phoenix/spaCy2/.env/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
